In [ ]:
### IMPORT ESSENTIAL LIBRARIES

import pandas as pd
import numpy as np
import time
import sys
sys.path.append("..") 

In [ ]:
#importing from sql file
import sqlite3
connection = sqlite3.connect("../db.sqlite3")
crsr = connection.cursor() 
crsr.execute("SELECT * FROM CountryIndicators")  
ans = crsr.fetchall()

In [ ]:
#Creating dataframe
df = pd.DataFrame(ans, columns=["CountryCode", "IndicatorCode", "Year", "Value" ])

In [ ]:
#looping all countries 
countries = df['CountryCode'].unique().tolist()
country_NON_GDP = ["ASM", "CUW", "FRO", "GUM", "PRK", "MNP", "SXM", "MAF", "TCA"]
for n in country_NON_GDP: 
    countries.remove(n)
countries

In [ ]:
indicators = df['IndicatorCode'].unique()

In [ ]:
len(indicators)

In [ ]:
#Creating a dictionary with all countries and looping it  
d = {}
for name in countries:
    d[name] = df.loc[df['CountryCode'] == name] 
    print(name)
d['ARB']

In [ ]:
d['ARB']['CountryCode'][1]

In [ ]:
from functools import reduce
#Creation of a dataframe of year 
data_years = pd.DataFrame(range(1960,2011), columns = ['Year'])
#to initialize df as data_years
df_final = data_years
#We want a dictionary of dictionaries from each country 
dcountries = {}
#to create an auxuliar dataframe
df_auxiliar = pd.DataFrame(columns = indicators)
Years = ["Year"]
CountryName = ["CountryName"]
df_aux = pd.DataFrame(columns = Years)
df_aux = pd.concat([df_aux, df_auxiliar], axis=1, join='inner')
df_aux.set_index('Year', inplace=True)
s = time.time()
for c in countries: 
    dcountries[c] = {}
for c in countries: 
        for names in indicators: # it would be the indicators
            dcountries[c][names] = d[c].loc[d[c]['IndicatorCode'] == names]
            dcountries[c][names] = dcountries[c][names].rename(columns = {'Value': '{}_val'.format(names),'IndicatorCode': '{}'.format(names) }) #to convert variables into strings
            df_final = reduce(lambda left,right: pd.merge(left,right,on='Year', how = 'outer'), [df_final, dcountries[c][names]]) #to merge de df 
            df_final = df_final[df_final.columns[pd.Series(df_final.columns).str.contains("Year|val")]]   #we choose the right variables
            df_final.set_index('Year', inplace=True) #to change the index preparaing the df like the model's example
            #print(names)
        
        df_final = reduce(lambda left,right: pd.merge(left,right,on='Year', how = 'outer'), [df_final, GDP_next_year(c)])
        CountryName = {'Country_name' : ([c]*51) ,'Year' : range(1960,2011)}
        df_country = pd.DataFrame(CountryName, columns = ['Country_name' ,'Year'])
        df_country.set_index('Year', inplace=True)
        df_final = pd.merge(df_country, df_final, on = 'Year', how = 'outer')
        df_final.to_csv("df_country_gradient/{}.csv".format(c))
        # TO ADD A COLUMS WHICH INIDCATES DE COUNTRY (C)
        df_aux = pd.concat([df_final,df_aux], ignore_index=False, sort=False) #change ignore_index 
        df_final = data_years
df_aux.sort_values("Year").to_csv("df_country_gradient/df_final.csv")

In [ ]:
c = 'ARB'
names = 'NY.GDP.MKTP.KD.ZG'
df_years_2011 = pd.DataFrame(range(1959,2010), columns = ['Year'])
data_years = pd.DataFrame(range(1960,2011), columns = ['Year'])
df_final = data_years

dcountries[c][names] = d[c].loc[d[c]['IndicatorCode'] == names]
dcountries[c][names] = dcountries[c][names].rename(columns = {'Value': '{}_val'.format(names),'IndicatorCode': '{}'.format(names) }) #to convert variables into strings
df_final = reduce(lambda left,right: pd.merge(left,right,on='Year', how = 'outer'), [df_final, dcountries[c][names]]) #to merge de df 
df_final = df_final[df_final.columns[pd.Series(df_final.columns).str.contains("Year|val")]]   #we choose the right variables
df_final = df_final.drop([0])
df_final['Year'] = df_years_2011['Year'] 
df_final.set_index('Year', inplace=True) #to change the index preparaing the df like the model's example
df_final

In [ ]:
def GDP_next_year(c): 
    names = 'NY.GDP.MKTP.KD.ZG'
    df_years_2011 = pd.DataFrame(range(1959,2010), columns = ['Year'])
    data_years = pd.DataFrame(range(1960,2011), columns = ['Year'])
    df_final = data_years

    dcountries[c][names] = d[c].loc[d[c]['IndicatorCode'] == names]
    dcountries[c][names] = dcountries[c][names].rename(columns = {'Value': '{}_val_next_year'.format(names),'IndicatorCode': '{}'.format(names) }) #to convert variables into strings
    df_final = reduce(lambda left,right: pd.merge(left,right,on='Year', how = 'outer'), [df_final, dcountries[c][names]]) #to merge de df 
    df_final = df_final[df_final.columns[pd.Series(df_final.columns).str.contains("Year|val")]]   #we choose the right variables
    df_final = df_final.drop([0])
    df_final['Year'] = df_years_2011['Year'] 
    df_final.set_index('Year', inplace=True) #to change the index preparaing the df like the model's example
    return df_final

In [ ]:
GDP_next_year('ARB')